## 1. Introduction

In this report, I will present the implementation of the MCTS-based adaptive stress testing (AST) method presented in [1] on the AutomotiveDrivingModels (ADM) simulator developed in https://github.com/sisl/AutomotiveDrivingModels.jl. The goal of this implementation is to test the models in the simulator by finding the cases where they fail, through exploiting any stochasticity in the environment. This can be done by putting the systems under test in a totally random environment, like in the random walker scenario shown in section 4. However, it is not of interest to explore these situations, as they do not offer any insight into real world cases. For that reason, other more deterministic models have been used with some perturbations. Another thing to note is that it is also not of interest to find cases where the system under test is not at fault, as they do not offer any insight on how to amend the system to avoid such situations. For that case, the last crosswalk scenario is explored. That being said, some of these scenarios are presented to demonstrate the ability of AST to find failure cases for ADM models.

## 2. Methodology
The main part of this implementation is to interface AST with ADM to find cases where vehicles engage in collisions or be in a near collision situation, either with each other or with a pedestrian. The AST method is interfaced with the simulator in the form of the following functions:
* **Initialize**: This function sets the simulator to its initial state.
* **Update**: This function updates the state of the simulator, by stepping it one time step. It returns the likelihood of the new state, whether the new state is an event, and the minimum distance between the vehicles.
* **IsTerminal**: This function returns whether an event happened or if the horizon has been reached.

The reward model used in the applied AST is as follows:
$$R(s_t,s_{t+1}) = \left\{
        \begin{array}{ll}
            0 & \quad s_t \in E, \\
            min_o(dist(p_e,p_o)) & \quad s_t \notin E, t\geq T \\
            log P(s_{t+1}\mid s_t)   & \quad s_t \notin E, t\lt T
        \end{array}
    \right.$$
where $E$ is the event state, $T$ is the horizon, $p_e$ is the test vehicle's position, and $p_o$ is the position of any other vehicle in the environment. This reward model enables the AST method to get closer to situations where a collision occures, since the distance between vehicles decrease monotonically until they collide.

The method has been applied to four scenarios, the first is a highway scenario, and the other three are crosswalk scenarios. I will explain in each section how each scenario is built, modeled, and what are the resulting behaviors.

## 3. Highway scenario

This scenario contains three vehicles, one of them is the system under test. The three vehicles are modeled as follows:
* Test vehicle: 
    * **Longitudinal model**: *IntelligentDriverModel*, this model reads the distances to the cars ahead in the lane and their speed, and tries to match the test car's longitudinal acceleration with the other car's speed.
    * **Lateral model**: *ProportionalLaneTracker*, uses a PD controller to keep the vehicle in the lane.
    * **Lane changing model**: *TimLaneChanger*, reads the distances to other cars along the lane, if they are slow, it reads the distances along other lanes, and switches if there is an available space.
* Dummy vehicles:
    * **Longitudinal model**: *IntelligentDriverModel*, same as above, but the resulting acceleration is not deterministic, and is perturbed by a gaussian noise: $a_{lon} \sim N(a_{lon},\sigma)$.
    * **Lateral model**: *ProportionalLaneTracker*, same as above, but the lateral acceleration is perturbed by gaussian noise: $a_{lat} \sim N(a_{lat},\sigma)$.

MCTS would exploit these perturbations in the dummy vehicles in order to find the edge case where a collision between the test vehicle and one of the dummy vehicles would occur. The expected outcome of this scenario is that a collision will occur, but it will only be the fault of the other vehicles, since the test vehicle's system is deterministic and will never break its rules.. The following figure shows the initial set up of the scenario:

<img src="highway.png">
**Figure 1**: The test vehicle shown in red, while the dummy vehicles shown in blue.

After running multiple simulations with different initial seeds, they all led to the same outcome, albeit with slight differences in the dummy cars' trajectories, but for all intents and purposes they can be considered the same. The following animation shows the outcome:

<img src="scene_seed_18_R_-207.569609.gif">
**Figure 2**: This scene has a log likelihood of -207.569609.


In this scene, the collision occurred at the rear of the test vehicle. Since the models only sense what is in front of them, they cannot detect and therefore avoid any rear threat.
This scenario was used to illustrate that functions described above can be implemented on the AutomotiveDrivingModels simulator, and that the MCTS adaptive stress testing method will find cases where an event will happen in the simulator. In the following section, the environment is changed to the crosswalk environment, where more interesting cases are explored.

## 4. Crosswalk Scenarios

Three crosswalk scenarios have been implemented, they all consisted of a test vehicle that has the same model as the test vehicle in the highway scenario, and a pedestrian. The model of the pedestrian is what distinguished these three scenarios.
The following figure shows the initial setup for the crosswalk scenarios:

<img src="crosswalk.PNG">
**Figure 3**: The initial crosswalk scene, the test car in red and the pedestrian in blue.


The car is chosen so close to the crosswalk in order to increase the likelihood of an event happening, if the car was further it is more likely that the pedestrian will cross the street before the arrival of the vehicle.

### Random walker

The first pedestrian model is a random walker that samples its longitudinal and lateral accelerations from a normal distribution:
* $a_{lon} \sim N(0,\sigma)$
* $a_{lat} \sim N(0,\sigma)$

This resulted in an erratic behavior of the pedestrian, as the acceleration and the direction can change rapidly. The expected outcome of this scenario is that MCTS would succeed in exploiting the erratic behavior to find scenes where the pedestrian will head for the car immediately, causing a collision. The following animation shows the outcome of this scenario:

<img src="scene_ped_rand_seed_3_R_-75.077362.gif">
**Figure 4**: This scene has a log likelihood of -75.077362.

Which shows that the collision was due to the pedestrian jumping on the car as it approached the crosswalk, MCTS led to this outcome since it was easy to lead the random walker through a series of uneven accelerations that minimized the runtime and therefore maximized the log likelihood.

### White noise acceleration

In the second pedestrian model I tried to make it more rigid and more realistic to eliminate the possibility of MCTS leading it towards easy unrealistic collisions like in the random walk model. In this model, the lateral acceleration is fixed to zero, so that the pedestrian will move only in one dimension. As for the longitudinal acceleration, it had a Gaussian model similar to the dummy cars in the highway scenario: $a_{lon} \sim N(a_{lon},\sigma)$. Now the state space is limited to cases where the collision happens only if the driver crosses the crosswalk while the pedestrian is there (i.e. no collisions outside or on the edge of the crosswalk, which are less likely). However, since the driver model used for the test car is a rule-based model that has perfect information, MCTS is expected to fail to find a collision, instead given the reward model used that favors situations that lead to minimum distance between the objects, it will lead to some interesting pedestrian behavior as it tries to put itself in front of the test vehicle. Several simulations were ran with different seeds, the one with the highest likelihood is shown below:

<img src="scene_ped_seed_3_R_-282.894483.gif">
**Figure 5**: This scene has a log likelihood of -282.894483.

In this scene, MCTS tries to find the sequence of actions that will lead to the minimum distance between the pedestrian and the vehicle when the horizon of the simulation is reached. That led to the pedestrian trying to stay stationed in its initial position, which required a series of accelerations and decelerations since it starts with nonzero initial velocity, and as the end of the horizon approaches it finally decides to cross so that by the endtime it is exactly in front of the vehicle. As for the vehicle's behavior, the crosswalk (when there is a pedestrian) presents an obstacle for the vehicle, and for that the only option that is available to the vehicle is to stop at the crosswalk until it is cleared. The jittering in the vehicle's direction is caused by a bug in the simulator. To solve the issue of the rigid car model, noise can be added to its sensor reading, or another more complex and stochastic model can be used. The next scenario shows the result of the former.

### Sensor noise

The third crosswalk scenario is similar to the second with a slight change in the test vehicle's model. Instead of the model getting perfect information, now there is a chance that the fore vehicle indicator will give false negatives, this was done by modifying the *track_longitudinal* function in the *IntelligentDriverModel*. Now there is a possibility that the test vehicle will cross the crosswalk while the pedestrian is crossing, and therefore cause a collision where the test vehicle is at fault. After running multiple simulations with different seeds, some of them retuned similar behavior as the second scenario, but now there are scenarios where there is a collision with the test vehicle being at fault. The following animation shows such scene:

<img src="scene_ped_noise1_seed_4_R_-33.101541.gif">
**Figure 6**: This scene has a log likelihood of -33.101541.

This scene shows a sequence of actions that led to the pedestrian crossing the road while the vehicle's sensors were giving false negatives about the presence of an obstacle. While it may look trivial, but this scene raises an important issue on how a vehicle should deal with noisy sensor measurements, espacially false negatives as they are harder to deal with.

## 5. Comments
In the presented scenarios, the adaptive stress testing method managed to drive the simulator either to a collision or to a state where the distance between the vehicles is minimized. The rule-based model with perfect information is shown to be robust against all of AST's attempts to drive it to failure. This is hardly surprising given that it is a deterministic system, while the stochasticity resided in the environment around it. However, as soon as stochasticity was added to it in the form of false negatives in one of the sensors' readings, the AST managed to find situations where this sensor malfunction would lead to a collision. I believe that given the current state of the simulator and the models it has, this was the best that can be exploited and tested under AST. Any more situations and scenarios to be tested mean that the simulator must first:
* Upgrade the IntelligentDriverModel to better deal with crosswalks.
* Upgrade the lane changing model, since it detected that there are no other lanes to switch to even though there is a left lane.

Further improvements can go beyond rule-based model to incorporate data driven models, such as the GAIL driver model developed in [2], or adapting the pedestrian social behaviors developed in [3,4] to the crosswalk environment.

## References
- [1] Lee, Ritchie, et al. "Adaptive stress testing of airborne collision avoidance systems." Digital Avionics Systems Conference (DASC), 2015 IEEE/AIAA 34th. IEEE, 2015.
- [2] Kuefler, Alex, et al. "Imitating driver behavior with generative adversarial networks." arXiv preprint arXiv:1701.06699 (2017).
- [3] Robicquet, Alexandre, et al. "Learning Social Etiquette: Human Trajectory Understanding In Crowded Scenes." European Conference on Computer Vision. Springer International Publishing, 2016.
- [4] Alahi, Alexandre, et al. "Social lstm: Human trajectory prediction in crowded spaces." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2016.

## Code

### Highway scenario

In [ ]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Vec
using Distributions
using AdaptiveStressTesting
import AdaptiveStressTesting.sample
using Records
using Interact

#########################################
# SimPrams is a type that holds the following simulation parameters:
# endtime: The horizon of AST.
# logging: Whether or not to record the scenes.
# car_loc: The locations of the vehicles.
#########################################
type SimParams
    endtime::Int64
    logging::Bool
    car_loc::Array{VecSE2, 1}
end

#########################################
# CarSim is a type that holds the following simulation attributes
# it is passed to every function and returned at the end to extract the 
# scenes:
# p: Simulation parameters descrbed above
# scene: The state of the simulator, contains all information about the vehicls.
# roadway: The environment of the simulation.
# models: A dictionary of the vehicles models, indexed by their ids.
# actions: A list of actions, the indexes match the cars' ids.
# initial: Contains the initial scene, used to initialize the simulator.
# states: A list that holds the sequence of states for each run
# t: Keeps track of the timesteps for AST.
#########################################
type CarSim
    p::SimParams
    scene::Scene             
    roadway::Roadway         
    models::Dict{Int, DriverModel}     
    actions::Array{Any,1}     
    initial::Scene
    states::Vector{Scene}        
    t::Int64
end

#########################################
# CarSim is a function that is used to instantiate the simulator
# takes in the simulation parameters and the vehicles models
# and returns the simulation instance
#########################################
function CarSim(params::SimParams, models::Dict{Int, DriverModel})

    scene = Scene()
    roadway = gen_stadium_roadway(2)

    car1 = Vehicle(VehicleState(params.car_loc[1], roadway, 10.0)
                , VehicleDef(),1)
    car2 = Vehicle(VehicleState(params.car_loc[2], roadway, 10.0),
                        VehicleDef(),2)
    car3 = Vehicle(VehicleState(params.car_loc[3], roadway, 10.0),
                        VehicleDef(),3)
    push!(scene, car1)
    push!(scene, car2)
    push!(scene, car3)
    initial = deepcopy(scene)
    actions = get_actions!(Array(Any, length(scene)), scene, roadway, models)

    CarSim(params, scene, roadway, models, actions, initial, Vector{Scene}(), 0)
end
#########################################
# initialize resets the simulator to the initial scene
# does not return any value, only changes the state of 
# the simulator.
#########################################
function initialize(sim::CarSim)
    sim.t = 0
    sim.scene = deepcopy(sim.initial)
    empty!(sim.states)
    if sim.p.logging
        push!(sim.states, deepcopy(sim.scene))
    end
end
#########################################
# updates steps the simulator, it returns the probability
# of the new state, an indicator whether an event happened 
# or not, and the minimum distance between the test vehicle
# and the dummy vehicles.
#########################################
function update(sim::CarSim)
    sim.t += 1

    get_actions!(sim.actions, sim.scene, sim.roadway, sim.models)
    tick!(sim.scene, sim.roadway, sim.actions, 0.1)
    prob = pdf(sim.models[2], sim.actions[2])*pdf(sim.models[3], sim.actions[3])
    loc1 = sim.scene[1].state.posG
    loc2 = sim.scene[2].state.posG
    loc3 = sim.scene[3].state.posG
    dista = [dist(convert(VecE2, loc1), convert(VecE2, loc3)), dist(convert(VecE2, loc1), convert(VecE2, loc2))]
    if sim.p.logging
        push!(sim.states, deepcopy(sim.scene))
    end
    return (prob, isevent(sim), minimum(dista))
end
#########################################
# isevent checks if the current state of the simulator is
# an event, and returns a boolean indicator.
#########################################
function isevent(sim::CarSim)
    event = false
    for i = 2:length(sim.scene)
        if is_colliding(sim.scene[1], sim.scene[i])
            event = true
        end
    end
    return event
end
#########################################
# isterminal indicates whether the current state is a 
# terminal state.
#########################################
function isterminal(sim::CarSim)
  isevent(sim) || sim.t >= sim.p.endtime
end
#########################################
# run_sim takes in the following AST parameters:
# maxtime: The horizon of AST.
# s: The standard deviation of the lateral and longitudinal accelerations
# of the dummy vehicles.
# d: The depth of MCTS.
# n: The number of iterations to run MCTs.
# seed: The random seed of AST.
# The simulation is instantiated as well as AST, stress_test is called
# then the final state of the simulator, the loglikelihood and the action sequence
# that lead to an event are all returned
#########################################
function run_sim(maxtime::Int64,s::Float64,d::Int64,n::Int64,seed::Int64)
    const MAXTIME = maxtime #sim endtime
    const RNG_LENGTH = 2
    timestep = 0.1
    sim_params = SimParams(MAXTIME, true, [VecSE2(30.0,-DEFAULT_LANE_WIDTH,0.0), VecSE2(15.0,0.0,0.0),
                                            VecSE2(45.0,-DEFAULT_LANE_WIDTH,0.0)])
    models = Dict{Int, DriverModel}()
    models[1] = Tim2DDriver(timestep)
    models[2] = LatLonSeparableDriver( # produces LatLonAccels
            ProportionalLaneTracker(σ = s), # lateral model
            IntelligentDriverModel(σ = s), # longitudinal model
    )
    models[3] = LatLonSeparableDriver( # produces LatLonAccels
            ProportionalLaneTracker(σ = s), # lateral model
            IntelligentDriverModel(σ = s), # longitudinal model
    )

    sim = CarSim(sim_params, models)


    ast_params = ASTParams(MAXTIME, RNG_LENGTH, seed, 0)
    ast = AdaptiveStressTest(ast_params, sim, initialize, update, isterminal)


    mcts_params = DPWParams()
    mcts_params.d = d
    mcts_params.ec = 100
    mcts_params.n = n
    mcts_params.k = 0.5
    mcts_params.alpha = 0.85
    mcts_params.kp = 1.0
    mcts_params.alphap = 0.0
    mcts_params.clear_nodes = true
    mcts_params.maxtime_s = realmax(Float64)
    mcts_params.rng_seed = UInt64(0)
    reward, action_seq = stress_test(ast, mcts_params)
    return sim, reward, action_seq
end

#########################################
# The rest of the code calls run_sim with multiple seeds and 
# saves the result of each run in a different animation gif file
#########################################

carcolors = Dict{Int,Colorant}()
carcolors[1] = colorant"red"
carcolors[2] = colorant"blue"
carcolors[3] = colorant"blue"

for seed = 1:100
    sim, reward, action_seq = run_sim(100,5.0,50,1000,seed)


    frames = Frames(MIME("image/png"), fps=10)
    for frame in 1 : length(sim.states)
        s = render(sim.states[frame], sim.roadway, cam=FitToContentCamera(), car_colors=carcolors)
        push!(frames, s)
    end
    println("creating gif...")
    write(@sprintf("scene_seed_%i.gif",seed), frames)
end



### Crosswalk scenarios

In [ ]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Vec
using Distributions
using AdaptiveStressTesting
import AdaptiveStressTesting.sample
using Records
#########################################
# SimPrams is a type that holds the following simulation parameters:
# endtime: The horizon of AST.
# logging: Whether or not to record the scenes.
#########################################
type SimParams
    endtime::Int64
    logging::Bool
end
#########################################
# Defining the crosswalk envrionment type
#########################################
type CrosswalkEnv
    roadway::Roadway
    crosswalk::Lane
end
#########################################
# CarSim is a type that holds the following simulation attributes
# it is passed to every function and returned at the end to extract the 
# scenes:
# p: Simulation parameters descrbed above
# scene: The state of the simulator, contains all information about the vehicls.
# roadway: The environment of the simulation.
# models: A dictionary of the vehicles models, indexed by their ids.
# actions: A list of actions, the indexes match the cars' ids.
# initial: Contains the initial scene, used to initialize the simulator.
# states: A list that holds the sequence of states for each run
# t: Keeps track of the timesteps for AST.
#########################################
type CarSim
    p::SimParams
    scene::Scene             # The scene contains onfo about cars
    roadway::CrosswalkEnv         # roadway, layout of the road
    models::Dict{Int, DriverModel}     # driver models for each car
    actions::Array{Any,1}     # actions for each car, MCTS controls actions[1]
    initial::Scene
    states::Vector{Scene}        # Array that holds all states
    t::Int64
end
#########################################
# Defining the action type for the pedestrian
#########################################
immutable LaneSpecificAccelLatLon
    a_lat::Float64
    a_lon::Float64
end
#########################################
# The new pedestrian type, initialized by an initial acceleration
# and standard deviation.
#########################################
type CrosswalkDriver <: DriverModel{LaneSpecificAccelLatLon}
    a::LaneSpecificAccelLatLon
    σ::Float64
    function CrosswalkDriver(
        a::LaneSpecificAccelLatLon,
        σ::Float64 = NaN
        )
        retval = new()
        retval.a = a
        retval.σ = σ
        retval
    end
end
#########################################
# This function is used to build the crosswalk environment.
#########################################
function append_to_curve!(target::Curve, newstuff::Curve)
    s_end = target[end].s
    for c in newstuff
        push!(target, CurvePt(c.pos, c.s+s_end, c.k, c.kd))
    end
    return target
end
#########################################
# Propagate is defined inside AutomotiveDrivingModels to propagate the 
# state of the vehicle further one step using the latest action. This function
# defines propagate for the pedestrian action type
#########################################
function AutomotiveDrivingModels.propagate(veh::Vehicle, action::LaneSpecificAccelLatLon, roadway::Roadway, Δt::Float64)
    lane_tag_orig = veh.state.posF.roadind.tag
    state = AutomotiveDrivingModels.propagate(veh, LatLonAccel(action.a_lat, action.a_lon), roadway, Δt)
    roadproj = proj(state.posG, roadway[lane_tag_orig], roadway, move_along_curves=false)
    retval = VehicleState(Frenet(roadproj, roadway), roadway, state.v)
    return retval
end

AutomotiveDrivingModels.get_name(model::CrosswalkDriver) = "CrosswalkDriver"
#########################################
# rand return the acceleration action, either perturbed or the same
# depending on the value of σ.
#########################################
function Base.rand(model::CrosswalkDriver)
    if isnan(model.σ) || model.σ ≤ 0.0
        model.a
    else
        LaneSpecificAccelLatLon(rand(Normal(model.a.a_lat, model.σ)),model.a.a_lon)
    end
end
#########################################
# Returns the probability of the acceleration action
#########################################
Distributions.pdf(model::CrosswalkDriver, a::LaneSpecificAccelLatLon) = pdf(Normal(model.a.a_lat,model.σ), a.a_lat)
#########################################
# This function is used in visualizign the crosswalk environment
#########################################
function AutoViz.render!(rendermodel::RenderModel, env::CrosswalkEnv)
    render!(rendermodel, env.roadway)

    curve = env.crosswalk.curve
    n = length(curve)
    pts = Array(Float64, 2, n)
    for (i,pt) in enumerate(curve)
        pts[1,i] = pt.pos.x
        pts[2,i] = pt.pos.y
    end

    add_instruction!(rendermodel, render_dashed_line, (pts, colorant"white", env.crosswalk.width, 1.0, 1.0, 0.0, Cairo.CAIRO_LINE_CAP_BUTT))
    return rendermodel
end
#########################################
# CarSim is a function that is used to instantiate the simulator
# takes in the simulation parameters and the vehicles models
# and returns the simulation instance. The extra code here is for 
# building the crosswalk environment, copied from the docs.
#########################################
function CarSim(params::SimParams, models::Dict{Int, DriverModel})
    r = 5.0
    A = VecSE2(0,DEFAULT_LANE_WIDTH,-π)
    B = VecSE2(0,0,0)
    C = VecSE2(r,-r,-π/2)
    D = VecSE2(r+DEFAULT_LANE_WIDTH,-r,π/2)
    E = VecSE2(2r+DEFAULT_LANE_WIDTH,0,0)
    F = VecSE2(2r+DEFAULT_LANE_WIDTH,DEFAULT_LANE_WIDTH,-π)
    roadway = Roadway()
    curve = gen_straight_curve(convert(VecE2, B+VecE2(-100,0)), convert(VecE2, B), 2)
    append_to_curve!(curve, gen_bezier_curve(B, C, 0.6r, 0.6r, 51)[2:end])
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, C), convert(VecE2, C+VecE2(0,-50.0)), 2))
    lane = Lane(LaneTag(length(roadway.segments)+1,1), curve)
    push!(roadway.segments, RoadSegment(lane.tag.segment, [Lane(lane)]))

    curve = gen_straight_curve(convert(VecE2, B+VecE2(-100,0)), convert(VecE2, B), 2)
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, B), convert(VecE2, E), 2)[2:end])
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, E), convert(VecE2, E+VecE2(50,0)), 2))
    lane = Lane(LaneTag(length(roadway.segments)+1,1), curve)
    push!(roadway.segments, RoadSegment(lane.tag.segment, [Lane(lane)]))

    curve = gen_straight_curve(convert(VecE2, F+VecE2(50,0)), convert(VecE2, F), 2)
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, F), convert(VecE2, A), 2)[2:end])
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, A), convert(VecE2, A+VecE2(-100,0)), 2))
    lane = Lane(LaneTag(length(roadway.segments)+1,1), curve)
    push!(roadway.segments, RoadSegment(lane.tag.segment, [Lane(lane)]))

    curve = gen_straight_curve(convert(VecE2, F+VecE2(50,0)), convert(VecE2, F), 2)
    append_to_curve!(curve, gen_bezier_curve(F, C, 0.9r, 0.9r, 51)[2:end])
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, C), convert(VecE2, C+VecE2(0,-50)), 2))
    lane = Lane(LaneTag(length(roadway.segments)+1,1), curve)
    push!(roadway.segments, RoadSegment(lane.tag.segment, [Lane(lane)]))

    curve = gen_straight_curve(convert(VecE2, D+VecE2(0,-50)), convert(VecE2, D), 2)
    append_to_curve!(curve, gen_bezier_curve(D, E, 0.6r, 0.6r, 51)[2:end])
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, E), convert(VecE2, E+VecE2(50,0)), 2))
    lane = Lane(LaneTag(length(roadway.segments)+1,1), curve)
    push!(roadway.segments, RoadSegment(lane.tag.segment, [Lane(lane)]))

    curve = gen_straight_curve(convert(VecE2, D+VecE2(0,-50)), convert(VecE2, D), 2)
    append_to_curve!(curve, gen_bezier_curve(D, A, 0.9r, 0.9r, 51)[2:end])
    append_to_curve!(curve, gen_straight_curve(convert(VecE2, A), convert(VecE2, A+VecE2(-100,0)), 2))
    lane = Lane(LaneTag(length(roadway.segments)+1,1), curve)
    push!(roadway.segments, RoadSegment(lane.tag.segment, [Lane(lane)]))

    cam = FitToContentCamera(0.0)
    crosswalk = Lane(LaneTag(1,1), gen_straight_curve(VecE2(-25.0, -25.0), VecE2(-25.0, 25.0), 2), width=8.0)
    env = CrosswalkEnv(roadway, crosswalk)
    
    #########################################

    PEDESTRIAN_DEF = VehicleDef(AgentClass.PEDESTRIAN, 1.0, 1.0)
    scene = Scene()
    cars = [Vehicle(VehicleState(VecSE2(-50.0,0,0), roadway.segments[1].lanes[1],roadway, 8.0),
             VehicleDef(), 1), Vehicle(VehicleState(VecSE2(-24.0,-5.0,π/2), env.crosswalk, roadway, 0.5),
             PEDESTRIAN_DEF, 2)]
    car1 = cars[1]
    car2 = cars[2]
    push!(scene, car1)
    push!(scene, car2)
    initial = deepcopy(scene)
    actions = get_actions!(Array(Any, length(scene)), scene, roadway, models)
    CarSim(params, scene, env, models, actions, initial, Vector{Scene}(), 0)
end
#########################################
# initialize resets the simulator to the initial scene
# does not return any value, only changes the state of 
# the simulator.
#########################################
function initialize(sim::CarSim)
    sim.t = 0
    sim.scene = deepcopy(sim.initial)
    empty!(sim.states)
    if sim.p.logging
        push!(sim.states, deepcopy(sim.scene))
    end
end
#########################################
# updates steps the simulator using the models of the vehicls, it returns the probability
# of the new state, an indicator whether an event happened 
# or not, and the distance between the test vehicle
# and the pedestrian.
#########################################
function update1(sim::CarSim)
    sim.t += 1
    get_actions!(sim.actions, sim.scene, sim.roadway.roadway, sim.models)
    tick!(sim.scene, sim.roadway.roadway, sim.actions, 0.1)
    prob = pdf(sim.models[2], sim.actions[2])
    loc1 = sim.scene[1].state.posG
    loc2 = sim.scene[2].state.posG
    dista = dist(convert(VecE2, loc1), convert(VecE2, loc2))
    if sim.p.logging
        push!(sim.states, deepcopy(sim.scene))
    end
    return (prob, isevent(sim), dista)
end
#########################################
# updates steps the simulator using the random walk model for 
# the pedestrian, it returns the probability
# of the new state, an indicator whether an event happened 
# or not, and the distance between the test vehicle
# and the pedestrian.
#########################################
function update2(sim::CarSim)
    sim.t += 1
    lat = rand(Normal(0.0, 5.0))
    lon = rand(Normal(0.0, 5.0))
    get_actions!(sim.actions, sim.scene, sim.roadway.roadway, sim.models)
    sim.actions[2] = LaneSpecificAccelLatLon(lat, lon)
    tick!(sim.scene, sim.roadway.roadway, sim.actions, 0.1)
    prob = pdf(Normal(0.0, 5.0), lat) * pdf(Normal(0.0, 5.0), lon)
    loc1 = sim.scene[1].state.posG
    loc2 = sim.scene[2].state.posG
    dista = dist(convert(VecE2, loc1), convert(VecE2, loc2))
    if sim.p.logging
        push!(sim.states, deepcopy(sim.scene))
    end
    return (prob, isevent(sim), dista)
end
#########################################
# isevent checks if the current state of the simulator is
# an event, and returns a boolean indicator.
#########################################
function isevent(sim::CarSim)
    is_colliding(sim.scene[1], sim.scene[2])
end
#########################################
# isterminal indicates whether the current state is a 
# terminal state.
#########################################
function isterminal(sim::CarSim)
  isevent(sim) || sim.t >= sim.p.endtime
end
#########################################
# run_sim takes in the following AST parameters:
# maxtime: The horizon of AST.
# s: The standard deviation of the lateral and longitudinal accelerations
# of the dummy vehicles.
# d: The depth of MCTS.
# n: The number of iterations to run MCTs.
# seed: The random seed of AST.
# The simulation is instantiated as well as AST, stress_test is called
# then the final state of the simulator, the loglikelihood and the action sequence
# that lead to an event are all returned.
# When simulating the random walker model, update2 is passed to AST,
# Otherwise, update1 is passed.
#########################################
function run_sim(maxtime::Int64,s::Float64,d::Int64,n::Int64,seed::Int64)
    const MAXTIME = maxtime #sim endtime
    const RNG_LENGTH = 2
    timestep = 0.1
    models = Dict{Int, DriverModel}()
    models[1] = Tim2DDriver(timestep, mlon = IntelligentDriverModel(), mlat = ProportionalLaneTracker())
    models[2] = CrosswalkDriver(LaneSpecificAccelLatLon(1.0,0.0), s)

    sim_params = SimParams(MAXTIME, true)

    sim = CarSim(sim_params, models)

    ast_params = ASTParams(MAXTIME, RNG_LENGTH, seed, 0)
    ast = AdaptiveStressTest(ast_params, sim, initialize, update1, isterminal)


    mcts_params = DPWParams()
    mcts_params.d = d
    mcts_params.ec = 100
    mcts_params.n = n
    mcts_params.k = 0.5
    mcts_params.alpha = 0.85
    mcts_params.kp = 1.0
    mcts_params.alphap = 0.0
    mcts_params.clear_nodes = true
    mcts_params.maxtime_s = realmax(Float64)
    mcts_params.rng_seed = UInt64(0)
    reward, action_seq = stress_test(ast, mcts_params)
    return sim, reward, action_seq

end

#########################################
# The rest of the code calls run_sim with multiple seeds and 
# saves the result of each run in a different animation gif file
#########################################

carcolors = Dict{Int,Colorant}()
carcolors[1] = colorant"red"
carcolors[2] = colorant"blue"

for seed = 1:10
    sim, reward, action_seq = run_sim(100,5.0,50,1000,seed)


    frames = Frames(MIME("image/png"), fps=10)
    for frame in 1 : length(sim.states)
        s = render(sim.states[frame], sim.roadway, cam=FitToContentCamera(), car_colors=carcolors)
        push!(frames, s)
    end
    println("creating gif...")
    write(@sprintf("scene_ped_noise2_seed_%i_R_%f.gif",seed,reward), frames)
end